## Ridge Regression (L2)
Es un tipo popular de regresión lineal regularizada que incluye una penalización L2. Esto tiene el efecto de reducir los coeficientes para aquellas variables de entrada que no contribuyen mucho a la tarea de predicción ya que solo se permite que las estimaciones de los parámetros se vuelvan grandes si hay una reducción proporcional en el error

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils.random import check_random_state
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

### Ejemplo Ridge Regression
Se aplicará Ridge Regression para predecir la distancia entre un par de puntos (x,y) y (x1,y1), uno de ellos constante.$$ d = \sqrt{(x1-x)^2 + (y1 - y)^2} $$

### Generar el conjunto de datos
Para generar el conjunto de datos se calculará la distancia entre un conjunto de 5000 puntos (x,y) aleatorios, así, el modelo de regresión se ajustará a una proporción del 80% del conjunto y el 20% restante estará destinado a la predicción.

In [2]:
def calcular_distancia(X,Y,XF,YF):
    return np.round(np.sqrt(np.power(XF-X,2)+np.power(YF-Y,2)),2)

XF = 5
YF = 12.5

rng = check_random_state(0)

X = np.round(rng.uniform(1, 11, 10000).reshape(5000, 2),2)
XFs=[[XF] for i in range(5000)]
YFs=[[YF] for i in range(5000)]
X=np.append(X, XFs, axis=1)
X=np.append(X, YFs, axis=1)
Y = calcular_distancia(X[:, 0],X[:, 1],XF,YF)
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

### Crear y ajustar el modelo
El parámetro alpha corresponde a qué tan fuerte será la regularización. La regularización mejora el condicionamiento del problema y reduce la varianza de las estimaciones. El valor 1.0 indica que se aplicará una penalidad completa.<br>
Número de divisiones (K-Fold) del conjunto de datos para el entrenamiento (n_splits=10) inicialmente tomará el prier subconjunto como validación y los demás de entrenamiento, luego el segundo bloque será de validación y el resto de prueba y así sucesivamente; en este caso el proceso se repetirá 3 veces (n_repeats=3).<br>

In [28]:
model = Ridge(alpha=1.0)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
scores = np.abs(scores)
print("Scores: ")
print(scores)
print()
print("Media de RMSE para los scores = ",np.mean(scores))

Scores: 
[0.40981546 0.38312434 0.44106025 0.39037346 0.42854711 0.40420448
 0.36709039 0.37242877 0.39601216 0.31145678 0.4099477  0.32260602
 0.39437493 0.41433177 0.38240475 0.41114641 0.3611454  0.33944632
 0.42410755 0.44214232 0.42428078 0.46972107 0.34799625 0.43431757
 0.37465527 0.37878345 0.39545227 0.35123778 0.35547861 0.37420971]

Media de RMSE para los scores =  0.39039663649084816


### Métricas
Se usará el GridSearch para buscar el valor optimizado de alpha, y luego, la métrica utilizada para medir el error sera RMSE.

In [29]:
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
resultado = search.fit(x_train,y_train)
print('RMSE para el entrenamiento: %.3f' % np.sqrt(np.abs(resultado.best_score_)))
print('Config: %s' % resultado.best_params_)

tst_predict = search.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test,tst_predict))
print("RMSE para las predicciones = ",rmse)


RMSE para el entrenamiento: 0.625
Config: {'alpha': 0.99}
RMSE para las predicciones =  0.6015138700543935
